In [15]:
import pandas as pd
from M4_data import prepare_M4_data
from M4_data import M4_parser
import numpy as np

In [16]:
X_train_df, y_train_df, X_test_df, y_naive2_df = prepare_M4_data('Daily', 500)

In [17]:
y_naive2_df

,unique_id,y,ds,y_hat_naive2
0,D1,2039.2,1972-10-04,2029.098331
1,D1,2035.0,1972-10-05,2027.546736
2,D1,2051.8,1972-10-06,2027.677330
3,D1,2061.8,1972-10-07,2027.289778
4,D1,2063.5,1972-10-08,2027.126240
...,...,...,...,...
6995,D500,9414.7,1981-07-08,9552.694018
6996,D500,9336.1,1981-07-09,9546.898891
6997,D500,9302.7,1981-07-10,9552.488906
6998,D500,9406.6,1981-07-11,9542.261801


In [25]:
def root_mean_squared_scaled_error(Y_panel, Y_hat_panel, Y_train_panel):
    """
    Receives test or validation predictions for models in Family for ids in Panel.
    Parameters:
        y_hat_panel : array-like or sparse matrix, shape (n_samples, n_features)
            Test or validation data, with column 'unique_id'.
        y : array_like, shape (n_samples, 1)
            Target values, with column 'unique_id'.
    Returns:
        loss_grouped: array_like, shape (unique_ids, 1)
            An array of floating point values, one for each unique_id.
    """
    # Models list
    #models_list = get_models_list(Y_hat_panel)
    # Sort and declare loss
    assert Y_panel.shape[0] == Y_hat_panel.shape[0]
    Y_panel = Y_panel.sort_values(by=["unique_id", "ds"]).reset_index(drop=True)
    Y_hat_panel = Y_hat_panel.sort_values(by=["unique_id", "ds"]).reset_index(drop=True)
    # Loss panel mse
    loss_panel = Y_hat_panel.loc[:, ["unique_id"]]
    loss_panel['y_lag'] = Y_train_panel.groupby('unique_id')['y'].shift(1)
    loss_panel['y_diff_train'] = np.square(Y_train_panel['y'] - loss_panel['y_lag'])
    
    print(loss_panel)
    model = 'y_hat_naive2'
    #for model in models_list:
    loss_panel_num = pd.DataFrame()
    loss_panel_num[model] = np.square(Y_panel["y"] - Y_hat_panel[model])
    loss_panel_num[model] = loss_panel_num[model].astype(float)
    
    collapse_vars =  ['y_diff_train']
    mean_loss_panel = loss_panel.groupby('unique_id')[collapse_vars].agg(np.nanmean).reset_index()
    #for model in models_list:
    mean_loss_panel['rmsse.' + model] = mean_loss_panel[model] / mean_loss_panel['y_diff_train']
    mean_loss_panel['rmsse.' + model] = np.sqrt(mean_loss_panel['rmsse.' + model])
    del loss_panel
    return mean_loss_panel

In [26]:
root_mean_squared_scaled_error(y_naive2_df[['unique_id','ds','y']], y_naive2_df[['unique_id','ds','y_hat_naive2']], y_train_df)

     unique_id   y_lag  y_diff_train
0           D1     NaN           NaN
1           D1  1017.1          4.84
2           D1  1019.3          5.29
3           D1  1017.0          4.84
4           D1  1019.2          0.25
...        ...     ...           ...
6995       D99  2506.2        222.01
6996       D99  2521.1         43.56
6997       D99  2527.7        841.00
6998       D99  2556.7        449.44
6999       D99  2535.5         60.84

[7000 rows x 3 columns]


KeyError: 'y_hat_naive2'